# **Outil de recherches sur les personnages tirés d'un document**

Cet outil permet de récupérer automatiquement les informations biographiques d'un personnage et de réaliser un travail statistique sur des informations des personnsages (date de naissance ou genre).

*Document d'entrée* : un fichier en format .txt contenant la liste des noms de personnages.

*Document de sortie* : un fichier txt avec la liste des identifiants Wikidata des personnages identifiés + un fichier txt avec le genre des personnages identifiés par Wikidata + un fichier JSON et un fichier Excel avec les informations bibliographiques concernant les personnages

In [ ]:
#@markdown # Connexion du notebook à son compte Google Drive et signalement du dossier de travail :

#@markdown - Lancer la cellule
#@markdown - Cliquer sur « Exécuter malgré tout » lors de l’apparition du message d’avertissement indiquant que le notebook n’a pas été créé par Google
#@markdown - Cliquer sur « Se connecter à Google Drive » lors de l’apparition du second message d’avertissement pour donner l’autorisation au notebook d’accéder à vos fichiers Google Drive
#@markdown - Choisir son compte Gmail puis cliquer sur « Autoriser »

from google.colab import drive
import re
import itertools
import json
import urllib.parse, urllib.request, json
import collections
from collections import Counter
import os
import pandas as pd

drive.mount('/content/drive/')
%cd /content/drive/My Drive/

#@markdown ###Indiquer le chemin vers le dossier de travail sur le Google Drive (si le dossier n'existe pas, il sera créé lors du lancement de la cellule) :
chemin_vers_le_dossier_de_travail = '/content/drive/My Drive/Personnages/'#@param {type:"string"}

if not os.path.exists(chemin_vers_le_dossier_de_travail):
      os.makedirs(chemin_vers_le_dossier_de_travail)
os.chdir(chemin_vers_le_dossier_de_travail)

In [5]:
#@markdown # Récupération des identifiants Wikidata des personnages contenus dans le fichier texte :
#@markdown ####Indiquer le chemin vers le fichier texte avec la liste des personnages à détecter :
chemin_liste_personnages = '/content/drive/My Drive/Personnages/liste_personnages.txt'#@param {type:"string"}
#@markdown ####Changer la langue si nécessaire (par défaut : français) :
langue = "fr" #@param {type:"string"}
#@markdown ####Si la détection des personnages n'est pas assez précise, descendre le seuil à 0.8 (moins de noms vont être identifiés). Dans le cas contraire, monter le seuil à 1.0 :
seuil = "0.8" #@param {type:"string"}

def CallWikifier(text, lang=langue, threshold=float(seuil)): ### Changer ici la langue et le seuil si nécessaire
    data = urllib.parse.urlencode([
        ("text", text), ("lang", lang),
        ("userKey", "nqvsutgqswvfmrcvyxjtopvpiukjtp"),
        ("pageRankSqThreshold", "%g" % threshold), ("applyPageRankSqThreshold", "true"),
        ("nTopDfValuesToIgnore", "200"), ("nWordsToIgnoreFromList", "200"),
        ("wikiDataClasses", "true"), ("wikiDataClassIds", "false"),
        ("support", "true"), ("ranges", "false"), ("minLinkFrequency", ""),
        ("includeCosines", "false"), ("maxMentionEntropy", "3")
        ])
    url = "http://www.wikifier.org/annotate-article"
# Appel de wikifier pour récupérer les informations :
    req = urllib.request.Request(url, data=data.encode("utf8"), method="POST")
    with urllib.request.urlopen(req, timeout = 100) as f:
        response = f.read()
        response = json.loads(response.decode("utf8"))
# Enregistrement des "ID" wikidata des personnages dans un fichier texte :
#@markdown ####Indiquer le nom du fichier créé avec la liste des identifiants Wikidata :
    nom_fichier_liste_identifiants = 'id_wikidata_personnages.txt' #@param {type:"string"}
    with open(nom_fichier_liste_identifiants, 'w') as outfile:
        for annotation in response["annotations"]:
          itemid = annotation["wikiDataItemId"]
          itemid2 = "".join([str(itemid)," "])
          outfile.write(itemid2)
          
with open(chemin_liste_personnages, "r") as myfile:
  liste=myfile.readlines()

CallWikifier(text=liste)

In [ ]:
#!pip uninstall wptools
!pip install wptools==0.1.6
import wptools

#@markdown # Récupération du genre de chaque personnage identifié avec Wikidata :

#@markdown Cette cellule récupère le genre dans lequel est classé le personnage dans Wikidata dans un fichier texte.

#@markdown ####Indiquer le nom du fichier texte avec la liste des identifiants Wikidata des personnages :
nom_fichier_liste_identifiants = 'id_wikidata_personnages.txt' #@param {type:"string"}

a_file = open(nom_fichier_liste_identifiants, "r")
list_genre=[]
#@markdown ####Indiquer le nom du fichier texte créé avec la liste de genre des personnages :
nom_fichier_genre = 'liste_perso_genre.txt' #@param {type:"string"}
with open(nom_fichier_genre, 'w') as outfilegenre:
  for line in a_file:
    for word in line.split(): 
      try:
        recherche_genre = wptools.page(wikibase=word, lang="fr")
        recherche_genre.WIKIPROPS['P21'] = 'gender'
        recherche_genre.get_wikidata()
        genre=recherche_genre.wikidata['gender']
      except KeyError as err:
        genre="None"
      genre2 = "".join([str(genre)," "])
      outfilegenre.write(genre2)

liste_genres = []
with open(nom_fichier_genre, "r") as f:
    for line in f:
        liste_genres.extend(line.split())
        print(liste_genres)

In [ ]:
!pip install --upgrade wptools
import wptools

#@markdown # Récupération d'informations bibliographiques sur les personnages :

#@markdown /!\ Attention avant d'exécuter cette cellule, une procédure doit être suivie :
#@markdown - Dans l'onglet "Exécution" cliquer sur "Redémarrer l'environnement d'exécution"
#@markdown - Exécuter la première cellule de ce notebook (celle qui connecte le notebook au Google Drive)
#@markdown - Enfin, exécuter cette cellule

#@markdown ####Indiquer le nom du fichier texte avec la liste des identifiants Wikidata des personnages :
nom_fichier_liste_identifiants = 'id_wikidata_personnages.txt' #@param {type:"string"}

#@markdown ####Indiquer le nom du fichier texte avec la liste des identifiants Wikidata des personnages :
nom_fichier_genre = 'liste_perso_genre.txt' #@param {type:"string"}

#@markdown ####Indiquer le nom du fichier en format JSON créé avec les informations sur les personnages :
fichier_json_infos_personnages = "donnees_personnages.json" #@param {type:"string"}

#@markdown ####Indiquer le nom du fichier en format XLSX créé avec les informations sur les personnages :
fichier_xlsx_infos_personnages = "informations_personnages.xlsx" #@param {type:"string"}

### Récupère les informations sur les personnages, créée un fichier json avec toutes les infos
donnees = {}
nom = []
date_naissance_search = []
date_deces_search = []
lieu_naissance_net = []
lieu_deces_net = []
genre = []
with open(nom_fichier_genre, "r") as f:
  with open(nom_fichier_liste_identifiants, "r") as a_file:
    with open(fichier_json_infos_personnages, "w") as fichier : 
        for line in a_file:
          for line2 in f:
            for word, sexe in zip(line.split(), line2.split()):
              genre = str(sexe)
              page = wptools.page(wikibase=word, lang="fr")
              page.get_wikidata()
              page.get_parse()
    ## Pour avoir toutes les informations :
              try:
                infobox = page.data['infobox']
                nom = infobox.get("nom")
                date_deces = infobox.get("date de décès")
                date_deces_search = re.findall(r'\b\d{3,4}\b', str(date_deces)) ### Conserver uniquement les années
                if len(date_deces_search) == 2:
                  date_deces_search = date_deces_search[0]
                else:
                    date_deces_search = date_deces_search
                date_naissance = infobox.get("date de naissance")
                date_naissance_search = re.findall(r'\b\d{3,4}\b', str(date_naissance)) ## Conserver uniquement les années
                if date_naissance_search:
                  date_naissance_search = date_naissance_search[0]
                lieu_naissance = infobox.get("lieu de naissance") 
                lieu_naissance2 = re.sub(r'([()[\]{}]|)', "",str(lieu_naissance))
                lieu_naissance3 = re.sub(r'(<br>)', "",str(lieu_naissance2))
                lieu_naissance_net = re.sub(r'({{-}})', "",str(lieu_naissance3))
                lieu_deces = infobox.get("lieu de décès")
                lieu_deces2 = re.sub(r'([()[\]{}]|)', "",str(lieu_deces))
                lieu_deces3 = re.sub(r'(<br>)', "",str(lieu_deces2))
                lieu_deces_net = re.sub(r'({{-}})', "",str(lieu_deces3))
              except AttributeError as ar:
                print("Ce n'est pas un personnage!")
              donnees[nom] = {"Date de naissance" : date_naissance_search, 
                                  "Date de deces" : date_deces_search,           
                                  "Lieu de naissance" : lieu_naissance_net,
                                  "Lieu de deces" : lieu_deces_net, 
                                  "Genre": genre}
                
        json.dump(donnees, fichier, ensure_ascii=False)

#Enregistrement de toutes les informations récupérées sous la forme d'un tableau Excel:
df = pd.DataFrame(donnees)
df2 = df.transpose()
df2.to_excel(fichier_xlsx_infos_personnages)

In [ ]:
#@markdown # Études statistiques sur les siècles de naissance des personnages :

#@markdown Cette cellule affiche dans l'ordre :
#@markdown - les dates de naissances des personnages identifiés
#@markdown - les siècles d'appartenance des personnages identifiés
#@markdown - le nombre de personnages identifiés par siècle

#@markdown Pour obtenir le dernier résultat, indiquer ici le siècle recherché sous la forme suivante, "19" pour le XIXe siècle : 
siecle_recherche=19#@param {type:"integer"}

#@markdown Les résultats s'affichent en dessous de la cellule.

keys = donnees[nom]["Date de naissance"]
liste_annees = []
fichier_json = fichier_json_infos_personnages
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_annees=value["Date de naissance"]
      liste_annees.append(test_annees)
  print(liste_annees)

def calcul_siecles(liste):
  liste_siecles=[]
  for annee in liste:
    annee_net = re.sub(r'[\[\]\']', "",str(annee))
    try:
      if (int(annee_net) <= 100):
        siecle = "1"
        liste_siecles.append(siecle)
      elif (int(annee_net) % 100 == 0):
        siecle=(annee_net // 100)
        liste_siecles.append(siecle)
      else:
        siecle=(int(annee_net) // 100 + 1)
        liste_siecles.append(siecle)
    except ValueError as valerr:
      pass
  print(liste_siecles) ## Affiche les siècles correspondant aux années
  nbr_siecle=liste_siecles.count(siecle_recherche)
  print(nbr_siecle)
  occurrences_siecles = collections.Counter(liste_siecles) ## Calcul du nombre d'occurences de chaque siècle
 
calcul_siecles(liste_annees)

In [ ]:
#@markdown # Étude statistique sur le genre des personnages :

#@markdown Cette cellule affiche dans l'ordre :
#@markdown - les genre des personnages identifiés 
#@markdown - le nombre de personnages référencés comme masculin, féminin ou "None" (lorsque l'information n'est pas donnée)
#@markdown - la proportion de chaque genre 

#@markdown Les résultats s'affichent en dessous de la cellule.

keys = donnees[nom]["Genre"]
liste_des_genres = []
fichier_json = fichier_json_infos_personnages
with open(fichier_json, "r") as f :
  contenu_json = json.load(f)
  pairs = contenu_json.items()
  for key, value in pairs:
      test_genres=value["Genre"]
      liste_des_genres.append(test_genres)
  print(liste_des_genres)

def proportion_genres(liste):
  c = Counter(liste)
  print(c) ## Affiche le nombre masculin / fémnin / None
  prop = [(i, c[i] / len(liste) * 100.0) for i in c]
  print(prop) ## Affiche la proportion de chaque
  
proportion_genres(liste_des_genres)